# Question 6. Build a Staging Model for FHV Data

Create a staging model for the For-Hire Vehicle (FHV) trip data for 2019.

Load the FHV trip data (https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/fhv) for 2019 into your data warehouse
Create a staging model stg_fhv_tripdata with these requirements:
Filter out records where dispatching_base_num IS NULL
Rename fields to match your project's naming conventions (e.g., PUlocationID → pickup_location_id)
What is the count of records in stg_fhv_tripdata?

- 42,084,899
- 43,244,693
- 22,998,722
- 44,112,187

## Objective

Load the FHV trip data for 2019 into DuckDB and create a staging model `stg_fhv_tripdata` using dbt with the following requirements:

- Filter out records where `dispatching_base_num IS NULL`
- Rename columns to match project naming conventions
- Count the number of records in the staging model

---

## Step 1: Download and Load FHV Data into DuckDB

I created a Python ingestion script similar to the yellow and green taxi ingestion process.

This script:

- Downloads all FHV CSV files for 2019
- Converts them to Parquet format using DuckDB
- Loads them into DuckDB under schema `prod`
- Creates table `prod.fhv_tripdata`

File path - [Ingest Data for Question 6](./data/taxi_rides_ny/ingest_data_for_q6.py)
Run the above file and it will fetch the FHV data and then we can proceed with some next steps

## Step 2: Define FHV Source in dbt

I added the FHV source to the models/staging/sources.yml file:

In [ ]:
- name: fhv_tripdata
  description: Raw FHV trip records
  loaded_at_field: pickup_datetime

Full source reference becomes:

database: taxi_rides_ny
schema: prod
table: fhv_tripdata

## Step 3: Create staging model stg_fhv_tripdata

Created file: [models/staging/stg_fhv_tripdata.sql](./data/taxi_rides_ny/models/staging/stg_fhv_tripdata.sql)
with following SQL:

In [ ]:
{{ config(materialized='view') }}

SELECT
    dispatching_base_num,
    pickup_datetime,
    dropOff_datetime,
    PUlocationID AS pickup_location_id,
    DOlocationID AS dropoff_location_id,
    SR_Flag AS sr_flag
FROM {{ source('raw', 'fhv_tripdata') }}
WHERE dispatching_base_num IS NOT NULL

This staging model:
- Renames columns to match naming conventions
- Filters invalid records
- Creates clean staging layer

---

## Step 4: Run dbt model

Command executed:

In [ ]:
dbt run --select stg_fhv_tripdata --target prod

And open DockDB CLI using 

In [ ]:
duckdb taxi_rides_ny.duckdb

and then run the below query

In [ ]:
SELECT COUNT(*)
FROM prod.stg_fhv_tripdata;

![Question 6](./images/Question6.png)

Answer: 43,244,693